## Train Test Split

### Test Set

In [ ]:
# def get_test_instances(test):
def get_test_instances(df):
    """
    This functions generates test instances using the Leave One Out Approach taking the maximum recommendation date
    """
    max_df = df.groupby("user_id_categorical").agg({"date": np.nanmax}).reset_index()
    rec_df = pd.merge(df, max_df, left_on=["user_id_categorical","date"], right_on=["user_id_categorical","date"])
    rec_df["user_id_categorical_new"] = rec_df["user_id_categorical"].map(swapped_users_dict)
    rec_df["app_id_categorical_new"] = rec_df["app_id_categorical"].map(swapped_games_dict)
    x_test = rec_df[["user_id_categorical_new", "app_id_categorical_new"]].values
    y_test = np.full((x_test.shape[0],1), 1)
    return x_test, y_test

x_test, y_test = get_test_instances(recommendations_sample)
print("x_test.shape",  x_test.shape)
print("y_test.shape",  y_test.shape)


C:\Users\riccardoricci\AppData\Local\Temp\ipykernel_24500\3121204030.py:6: FutureWarning: The provided callable <function nanmax at 0x000002A7FF7E6AF0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  max_df = df.groupby("user_id_categorical").agg({"date": np.nanmax}).reset_index()


x_test.shape (6533, 2)
y_test.shape (6533, 1)


In [ ]:
x_test_dict = dict(x_test)

### Get Training Istances

Including negative samples

In [ ]:
train = user_game_matrix_k_rec[1:,1:].todok()
num_users, num_items = train.shape
print("Number of Users:", '{0:,.0f}'.format(num_users))
print("Number of Games:", '{0:,.0f}'.format(num_items))


Number of Users: 4,916
Number of Games: 17,108


In [ ]:
def get_train_instances(train, num_negatives):
    """
    This function generates training instances. For each user in the training data,
    I should have users who recommended games and users who do not recommend games
    """
    user_input, item_input, labels = [],[],[]
    for (u, i) in train.keys():
        # positive instance
        if (u,i) not in x_test_dict.keys(): # The test instance must not be included
            user_input.append(u)
            item_input.append(i)
            labels.append(1)
            # negative instances
            item_selected = [] # to avoid duplicates
            for t in range(num_negatives):
                j = np.random.randint(num_items)
                while ((u, j) in train.keys()) or ((u,j) in item_selected) or ((u,j) in x_test_dict.keys()):
                    j = np.random.randint(num_items)
                user_input.append(u)
                item_input.append(j)
                labels.append(0)
                item_selected.append((u,j))
        
    p = np.vstack([user_input, item_input, labels]).T
    p = np.unique(p, axis=0)
    x_train = p[:,:2]
    y_train = p[:, 2].reshape(-1, 1)
    return x_train, y_train

x_train, y_train = get_train_instances(train, 4)
print("x_train.shape",  x_train.shape)
print("y_train.shape",  x_train.shape)


x_train.shape (924063, 2)
y_train.shape (924063, 2)
